# Convert sql murder mystery to cypher!

https://mystery.knightlab.com/

In [1]:
#%pip install kuzu pandas

In [2]:
import kuzu
import sqlite3
import re
import shutil

sq = sqlite3.connect("sql.db")

db_path = '/tmp/foobar2'
shutil.rmtree(db_path, ignore_errors=True)
db = kuzu.Database(db_path)
conn = kuzu.Connection(db)

In [3]:
t_raw = sq.execute("select tbl_name, sql from sqlite_schema where type = 'table'").fetchall()
print(t_raw[0])

('crime_scene_report', 'CREATE TABLE crime_scene_report (\n        date integer,\n        type text,\n        description text,\n        city text\n    )')


In [4]:
def sql_to_schema(tname, sql):
    sql = sql.replace("\n","")
    rest = sql.split("(",1)[1].replace("("," ").replace(")"," ")
    rest = re.sub(r'\s+', ' ', rest)
    tbl = {'name': tname, 'fields': [], 'primary_key': None, 'foreign_key': None}
    for item in rest.split(","):
        f = item.strip().split()
        name = f[0]
        if name == "FOREIGN":
            fk_f = f[2]
            fk_t = f[4]
            fk_r = f[5]
            tbl['foreign_key'] = (fk_f, fk_t, fk_r)
            continue
        if 'PRIMARY' in f:
            tbl['primary_key'] = name
        tbl['fields'].append((f[0], f[1]))
    return tbl
    
schema = [sql_to_schema(tname, sql) for tname,sql in t_raw]
#schema

In [5]:

def cleanup(t):
    t = t.lower()
    if t in ['varchar','text','char']:
        return 'string'
    if t in ['bigint','integer']:
        return 'int64'
    return t
    
gschema = []
for ot in schema:
    t = ot.copy()
    if t['primary_key'] is None:
        t['fields'] = [('id','serial')] + t['fields']
        t['primary_key'] = 'id'
    t['fields'] = [(n,cleanup(t)) for n,t in t['fields']]
    gschema.append(t)

for t in gschema:
    fieldstr = ",".join([f"{n} {t}" for n,t in t['fields']] + [f"primary key ({t['primary_key']})"])
    cypher = f"create node table {t['name']} ({fieldstr});"
    print(cypher)
    conn.execute(cypher)


create node table crime_scene_report (id serial,date int64,type string,description string,city string,primary key (id));
create node table drivers_license (id int64,age int64,height int64,eye_color string,hair_color string,gender string,plate_number string,car_make string,car_model string,primary key (id));
create node table facebook_event_checkin (id serial,person_id int64,event_id int64,event_name string,date int64,primary key (id));
create node table interview (id serial,person_id int64,transcript string,primary key (id));
create node table get_fit_now_member (id string,person_id int64,name string,membership_start_date int64,membership_status string,primary key (id));
create node table get_fit_now_check_in (id serial,membership_id string,check_in_date int64,check_in_time int64,check_out_time int64,primary key (id));
create node table solution (id serial,user int64,value string,primary key (id));
create node table income (ssn string,annual_income int64,primary key (ssn));
create node

In [6]:
tuple([2,23])

(2, 23)

In [7]:
### COPY IN NODE DATA
import csv

def clean_cols(t):
    return tuple([c.replace("\n"," ").replace('"',"'") if type(c) == str else c for c in t])
    
for st,gt in zip(schema,gschema):
    with open("tmp.csv","w") as f:
        w = csv.writer(f)
        cols = [cn for cn,_ in st['fields']]
        w.writerow(cols)
        rows = sq.execute(f"select {','.join(cols)} from {st['name']}").fetchall()
        for row in rows:
            clean_row = clean_cols(row)
            w.writerow(clean_row)
    conn.execute(f"copy {gt['name']} from 'tmp.csv' (header=true, parallel=false)")
        

In [8]:
#conn.execute("match (i:income) return *").get_as_df()

In [9]:
cy = """
create rel table has_license (from person to drivers_license);
create rel table has_income (from person to income);
create rel table has_interview (from person to interview);
create rel table has_checkin (from get_fit_now_member to get_fit_now_check_in);
create rel table went_to_event (from person to facebook_event_checkin);
create rel table is_member (from person to get_fit_now_member);
"""
conn.execute(cy)

In [10]:
cy = """
match (p:person), (d:drivers_license) where p.license_id = d.id
create (p)-[:has_license]->(d);
match (p:person), (i:income) where p.ssn = i.ssn
create (p)-[:has_income]->(i);
match (p:person), (i:interview) where p.id = i.person_id
create (p)-[:has_interview]->(i);
match (p:person), (m:get_fit_now_member) where p.id = m.person_id
create (p)-[:is_member]->(m);
match (p:person), (e:facebook_event_checkin) where p.id = e.person_id
create (p)-[:went_to_event]->(e);
match (m:get_fit_now_member), (c:get_fit_now_check_in) where c.membership_id = m.id
create (m)-[:has_checkin]->(c);
"""
conn.execute(cy)

In [11]:
def cyph(s):
    return conn.execute(s).get_as_df()


In [12]:
cyph("""match (i:income) return count(*)""")

,COUNT_STAR()
0,7511


In [16]:
df = cyph("""match (r:crime_scene_report) where r.type = 'murder' and r.date = 20180115 return r.*""")
df

,r.id,r.date,r.type,r.description,r.city
0,1,20180115,murder,Life? Dont talk to me about life.,Albany
1,2,20180115,murder,"Mama, I killed a man, put a gun against his he...",Reno
2,1227,20180115,murder,Security footage shows that there were 2 witne...,SQL City


In [25]:
df.loc[df['r.id'] == 1227, 'r.description'].values

array(["Security footage shows that there were 2 witnesses. The first witness lives at the last house on 'Northwestern Dr'. The second witness, named Annabel, lives somewhere on 'Franklin Ave'."],
      dtype=object)

In [28]:
cyph("""
match (p:person) where p.address_street_name = 'Northwestern Dr'
return p.*
order by p.address_number desc
limit 3
""")

,p.id,p.name,p.license_id,p.address_number,p.address_street_name,p.ssn
0,14887,Morty Schapiro,118009,4919,Northwestern Dr,111564949
1,17729,Lasonya Wildey,439686,3824,Northwestern Dr,917817122
2,53890,Sophie Tiberio,957671,3755,Northwestern Dr,442830147


In [32]:
cyph("""
match (p:person {id: 14887})-[:has_interview]->(i:interview)
return i.transcript
""").values

array([["I heard a gunshot and then saw a man run out. He had a 'Get Fit Now Gym' bag. The membership number on the bag started with '48Z'. Only gold members have those bags. The man got into a car with a plate that included 'H42W'."]],
      dtype=object)